In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar("dog")

[('cat', 0.8798075318336487),
 ('dogs', 0.8344309329986572),
 ('pet', 0.7449564337730408),
 ('puppy', 0.7236376404762268),
 ('horse', 0.7109654545783997),
 ('animal', 0.6817063093185425),
 ('pig', 0.655417263507843),
 ('boy', 0.6545307636260986),
 ('cats', 0.6471932530403137),
 ('rabbit', 0.6468629837036133)]

In [7]:
glove_wiki_vectors.most_similar("cat")

[('dog', 0.8798074722290039),
 ('rabbit', 0.7424426078796387),
 ('cats', 0.7323004007339478),
 ('monkey', 0.7288709878921509),
 ('pet', 0.7190139889717102),
 ('dogs', 0.7163872718811035),
 ('mouse', 0.6915251016616821),
 ('puppy', 0.6800068616867065),
 ('rat', 0.6641027331352234),
 ('spider', 0.650113582611084)]

In [8]:
glove_wiki_vectors.most_similar("cheesecake")

[('chocolate', 0.7293441891670227),
 ('brownies', 0.6957851052284241),
 ('custard', 0.6954904198646545),
 ('caramel', 0.6932608485221863),
 ('cookies', 0.6721082329750061),
 ('cake', 0.6698510646820068),
 ('pie', 0.6569927334785461),
 ('pecan', 0.6563494205474854),
 ('dessert', 0.6547815799713135),
 ('cream', 0.6479873657226562)]

In [9]:
glove_wiki_vectors.most_similar("coffee")

[('tea', 0.7732690572738647),
 ('drinks', 0.7287518978118896),
 ('beer', 0.7253386378288269),
 ('cocoa', 0.7026590704917908),
 ('wine', 0.7002726793289185),
 ('drink', 0.6990923881530762),
 ('corn', 0.6825441718101501),
 ('sugar', 0.6775091886520386),
 ('bread', 0.6727856397628784),
 ('fruit', 0.667149007320404)]

In [10]:
glove_wiki_vectors.most_similar("matcha")

[('pu-erh', 0.6054583191871643),
 ('temi', 0.590826690196991),
 ('rooibos', 0.581093966960907),
 ('lapsang', 0.5794644355773926),
 ('chanoyu', 0.5764945149421692),
 ('tarnee', 0.5737316608428955),
 ('datil', 0.5632052421569824),
 ('catechins', 0.5601311922073364),
 ('jewelweed', 0.5601010322570801),
 ('kirtle', 0.5553528666496277)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for (w1, w2) in word_pairs:
    print(
        "Cosine similarity between %s and %s = %0.3f"
        % (w1, w2, glove_wiki_vectors.similarity(w1, w2))
    )

Cosine similarity between coast and shore = 0.700
Cosine similarity between clothes and closet = 0.546
Cosine similarity between old and new = 0.643
Cosine similarity between smart and intelligent = 0.755
Cosine similarity between dog and cat = 0.880
Cosine similarity between tree and lawyer = 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print("%s is represented in glove_wiki_vectors" %(word))
    else:
        print("%s is NOT represented in glove_wiki_vectors" %(word))

covididiot is NOT represented in glove_wiki_vectors
fomo is NOT represented in glove_wiki_vectors
frenemies is represented in glove_wiki_vectors
anthropause is NOT represented in glove_wiki_vectors
photobomb is NOT represented in glove_wiki_vectors
selfie is NOT represented in glove_wiki_vectors
pxg is NOT represented in glove_wiki_vectors
pacg is NOT represented in glove_wiki_vectors
cct is represented in glove_wiki_vectors
escc is represented in glove_wiki_vectors


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283244

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
analogy("man", "intelligent", "woman")

man : intelligent :: woman : ?


,Analogy word,Score
0,articulate,0.627857
1,thoughtful,0.616820
2,smart,0.587221
3,strong-willed,0.568862
4,vivacious,0.550627
5,opinionated,0.547024
6,perceptive,0.545705
7,inquisitive,0.544539
8,energetic,0.533960
9,literate,0.533894


In [20]:
analogy("man", "gentle", "woman")

man : gentle :: woman : ?


,Analogy word,Score
0,sensuous,0.601437
1,lovely,0.589904
2,sassy,0.580544
3,playful,0.580543
4,sultry,0.566077
5,soothing,0.564392
6,breezy,0.560906
7,demure,0.560848
8,vivacious,0.553936
9,refreshing,0.550070


In [21]:
glove_wiki_vectors.similarity("woman", "beautiful"), glove_wiki_vectors.similarity(
    "man", "beautiful"
)

(0.5252884, 0.5490399)

In [22]:
glove_wiki_vectors.similarity("woman", "engineer"), glove_wiki_vectors.similarity(
    "man", "engineer"
)

(0.33403102, 0.4299851)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. These embeddings appear to exhibit less bias compared to the GoogleNews pre-trained embeddings. This improvement can be attributed to modern methods for computing word embeddings, which incorporate debiasing techniques.
2. Using biased embeddings in real-world applications can have significant consequences. For example, an AI-powered hiring tool trained on biased embeddings might rank male candidates higher for roles like engineer or doctor while favoring female candidates for nursing or caregiving roles, perpetuating workplace inequality.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [26]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [27]:
dummy_df = pd.DataFrame(df[:20])
pd.DataFrame(dummy_df['text'])

,text
0,"You know, I was reading 18 U.S.C. 922 and some..."
1,\n\n\nIt's not a bad question: I don't have an...
2,"\nActuallay I don't, but on the other hand I d..."
3,"The following problem is really bugging me,\na..."
4,\n\n This is the latest from UPI \n\n For...
5,"Hi,\n I'd like to subscribe to Leadership Ma..."
6,"\n\nVery simple.\n\n""X-Soviet Armenian governm..."
7,\nNo no no!!! It's a squid! Keep the traditi...
8,From: Center for Policy Research <cpr>\nSubjec...
9,\n The OpenLook window manager source is avai...


In [28]:
# ref from lecture 18 

def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    

    # print("Named entities:\n", [(ent.text, ent.label_) for ent in doc.ents])

    clean_text = []
    tokens = [token for token in doc]  
    # print("\nTokens: "analogy, tokens)

    # Accessing lemma
    lemmas = [token.lemma_ for token in doc]
    # print("\nLemmas: ", lemmas)

    for token in doc:

        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
        if token.like_num:
            clean_text.append("NUM")

    # displacy.render(doc, style="ent")

    # print("Named entities:\n", [(ent.text, ent.label_) for ent in doc.ents])

    return " ".join(clean_text)

In [29]:
 # [ preprocess(text) for text in nlp.pipe(dummy_df['text'])]

In [30]:
dummy_df["text_pp"] = [preprocess(text) for text in nlp.pipe(dummy_df['text'])]
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df['text'])]


In [31]:
# dummy_df["named_ents"] = [preprocess(doc) for doc in nlp.pipe(dummy_df["text"], batch_size=50)]

In [32]:
# from spacy import displacy



# displacy.render(doc, style="ent")
# # Text and label of named entity span
# print("Named entities:\n", [(ent.text, ent.label_) for ent in doc.ents])
# print("\nORG means: ", spacy.explain("ORG"))
# print("GPE means: ", spacy.explain("GPE"))

In [33]:
# print(dummy_df['text'][0])
# print(dummy_df['text_pp'][0])


In [34]:
# comment out when using original df 
# from sklearn.feature_extraction.text import CountVectorizer
# vec = CountVectorizer(stop_words="english")
# X = vec.fit_transform(dummy_df["text_pp"])
# X


In [35]:
# dummy_df.iloc[2:6]

In [36]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read NUM u.s.c. 922 NUM sence wonder help...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 ----------...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure need correct...


In [37]:
# dummy_df.iloc[0]['text']

In [38]:
# df.iloc[0]['text']

In [39]:
# document_topics[0]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

* Lowercasing - to make words consistent for processing. i.e. 'name' and 'Name' are the same
* removing stopwords - Remove really common english words that won't contribute to topic modeling due to being so commonly used. (i.e. the, and, is)
* POS filtering - remove additional filler words like propositions, pronouns, etc that don't have much relevance to the overall meaning of a document. 
* Replace numbers with 'NUM' so that they don't impact topic modeling process. i.e. we don't want a category of just numbers.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [40]:
# from sklearn.feature_extraction.text import CountVectorizer
# vec = CountVectorizer(stop_words="english")
# X = vec.fit_transform(dummy_df["text_pp"])
# X


In [41]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words="english")
X = vec.fit_transform(df["text_pp"])
X


<4563x39306 sparse matrix of type '<class 'numpy.int64'>'
	with 275024 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
lda.fit(X) 
document_topics = lda.transform(X)

lda.components_

array([[1.63196995e+02, 1.73439615e+02, 1.25110310e-01, ...,
        1.25000000e-01, 1.25058070e-01, 1.25433862e-01],
       [1.09379366e+01, 5.42016867e+01, 2.12250691e+00, ...,
        1.25000000e-01, 1.25000001e-01, 1.25000004e-01],
       [1.25186953e-01, 5.67560126e+01, 1.26443806e-01, ...,
        1.25000000e-01, 1.25078245e-01, 1.25000006e-01],
       ...,
       [1.13326005e+00, 1.25054828e-01, 1.25000010e-01, ...,
        1.12500000e+00, 1.12478302e+00, 1.25000032e-01],
       [2.38046978e+00, 1.08089941e+02, 1.25161170e-01, ...,
        1.25000000e-01, 1.25000002e-01, 1.12443973e+00],
       [4.37481816e+00, 3.41662228e+00, 1.25758889e-01, ...,
        1.25000000e-01, 1.25000001e-01, 1.25126331e-01]])

In [43]:
print("Vocabulary size:", len(vec.get_feature_names_out()))

print("lda.components_.shape: {}".format(lda.components_.shape))

Vocabulary size: 39306
lda.components_.shape: (8, 39306)


In [44]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

In [45]:
feature_names = np.array(vec.get_feature_names_out())
feature_names

array(['00', '000', '0000', ..., 'zz', 'zzzzzz', 'zzzzzzt'], dtype=object)

In [46]:
import mglearn
mglearn.tools.print_topics(
    topics=range(8),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=4,
    n_words=15,
)


topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           jews          
game          file          say           num           
team          image         people        israeli       
play          edu           know          people        
year          window        come          right         
player        use           think         israel        
win           program       time          think         
season        available     kill          state         
good          server        tell          jewish        
hockey        run           armenians     point         
league        display       armenian      claim         
period        include       want          nazi          
55            work          israel        war           
new           widget        happen        know          
run           version       start         human         


topic 4       topic 5       t

In [47]:
document_topics[0]

array([0.23754225, 0.00208622, 0.00208723, 0.11535271, 0.00208456,
       0.00208336, 0.48630431, 0.15245937])

In [48]:
df.iloc[0]['text_pp']

'know read NUM u.s.c. 922 NUM sence wonder help u.s.c. 922 NUM NUM provide paragraph NUM shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute 922 NUM enforce individual tell government tell possess pass law law know law constitional go court go court run mill guilty jail claim right possess tell supreme court right'

In [49]:
df.iloc[1]['text_pp'] 

'bad question ref list algorithm think bit hard NUM NUM point find plane define circle lie algorithm NUM find center circle line pass center perpendicular plane NUM point pass center sphere NUM repeat unused point NUM original point give NUM different line pass sphere origin interection center sphere NUM radius easy compute distance center original point leave math workable algorithm alternate method pair point plane form perpendicular bisector line segment pair contain center sphere NUM pair form NUM plane intersect point easy implement'

In [50]:
document_topics[1]

array([0.00154642, 0.82367806, 0.00154645, 0.1670488 , 0.0015452 ,
       0.00154336, 0.00154575, 0.00154596])

In [51]:
df.iloc[2]['text'] 

"\nActuallay I don't, but on the other hand I don't support the idea of having\none newsgroup for every aspect of graphics programming as proposed by Brian,\nin his reply to my original posting.\nI would suggest a looser structure more like a comp.graphics.programmer,\ncomp.graphics.hw_and_sw\nThe reason for making as few groups as possible is for the same reason you\nsay we shouldn't spilt up, not to get to few postings every day.\nI takes to much time to browse through all postings just to find two or \nthree I'm interested in.\n\nI understand and agree when you say you want all aspects of graphics in one\nmeeting. I agree to some extension. I see news as a forum to exchange ideas,\nhelp others or to be helped. I think this is difficult to achive if there\nare so many different things in one meeting.\n\nGood evening netters|-)"

In [52]:
np.argmax(document_topics[2])

7

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [53]:
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")

Topic 0: num, game, team, play, year, player, win, season, good, hockey
Topic 1: num, file, image, edu, window, use, program, available, server, run
Topic 2: num, say, people, know, come, think, time, kill, tell, armenians
Topic 3: jews, num, israeli, people, right, israel, think, state, jewish, point
Topic 4: num, entry, file, output, program, section, line, char, rule, build
Topic 5: libxmu, captain, xmu, adam, trade, edu, doug, let, player, symbol
Topic 6: gun, num, law, people, weapon, state, right, firearm, government, crime
Topic 7: num, god, know, think, people, believe, jesus, say, time, good


In [54]:
topic_labels = {
    0: "Hockey + Sports",
    1: "Software Systems",
    2: "Armenian Genocide/Conflict",
    3: "Israel and Jewish Discussion",
    4: "Programming",
    5: "Software Libraries",
    6: "Politics",
    7: "Religion",
}


In [55]:
# Display assigned labels for verification
for topic_idx, label in topic_labels.items():
    print(f"Topic {topic_idx}: {label}")

Topic 0: Hockey + Sports
Topic 1: Software Systems
Topic 2: Armenian Genocide/Conflict
Topic 3: Israel and Jewish Discussion
Topic 4: Programming
Topic 5: Software Libraries
Topic 6: Politics
Topic 7: Religion


In [56]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read NUM u.s.c. 922 NUM sence wonder help...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 ----------...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure need correct...


In [57]:
topic_probabilities = pd.DataFrame(document_topics, columns=topic_labels)
topic_probabilities = topic_probabilities.rename(columns=topic_labels)
topic_probabilities

,Hockey + Sports,Software Systems,Armenian Genocide/Conflict,Israel and Jewish Discussion,Programming,Software Libraries,Politics,Religion
0,0.237542,0.002086,0.002087,0.115353,0.002085,0.002083,0.486304,0.152459
1,0.001546,0.823678,0.001546,0.167049,0.001545,0.001543,0.001546,0.001546
2,0.002087,0.280636,0.002087,0.002087,0.002085,0.002084,0.002086,0.706849
3,0.004169,0.384478,0.004171,0.004169,0.004170,0.004167,0.004168,0.590508
4,0.002908,0.046909,0.935636,0.002911,0.002909,0.002907,0.002911,0.002909
...,...,...,...,...,...,...,...,...
4558,0.004168,0.620765,0.004174,0.004171,0.004171,0.004167,0.004169,0.354215
4559,0.000023,0.999840,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023
4560,0.002605,0.002606,0.183337,0.002609,0.086464,0.002605,0.717164,0.002610
4561,0.003128,0.776198,0.003129,0.003126,0.205040,0.003126,0.003127,0.003127


In [58]:
df_combined = pd.concat([topic_probabilities, df], axis=1)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [59]:
df_combined.head()

,Hockey + Sports,Software Systems,Armenian Genocide/Conflict,Israel and Jewish Discussion,Programming,Software Libraries,Politics,Religion,text,target,target_name,text_pp
0,0.237542,0.002086,0.002087,0.115353,0.002085,0.002083,0.486304,0.152459,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read NUM u.s.c. 922 NUM sence wonder help...
1,0.001546,0.823678,0.001546,0.167049,0.001545,0.001543,0.001546,0.001546,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,0.002087,0.280636,0.002087,0.002087,0.002085,0.002084,0.002086,0.706849,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,0.004169,0.384478,0.004171,0.004169,0.004170,0.004167,0.004168,0.590508,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create NUM ...
4,0.002908,0.046909,0.935636,0.002911,0.002909,0.002907,0.002911,0.002909,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


From these first 5 rows of the document-topic probability assignments, it is clear that the LDA model assigns each document to a specific topic based on the highest probability (the predicted topic). In most cases, the probability for the most likely topic is much higher than for the other topics, which indicates that the model is confident in its assignment. However, the confidence in the predicted topic is relatively lower for row number 2 even though it is still more confident of the predicted topic than the other topics, this may suggest that the document may contain content that overlaps with multiple topics and leave room for improvement. Possible asjustments that could increase improvement are adjusting hyperparameters like max_iterations and n_components, or preprocess the text further. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6


1. content-based filtering gathers information about users mainly through the items they have interacted with. They are more likely to reccomend certain item to users based on their history of likes, dislikes, and ratings with regards to the item or content.

2. Collaborative filtering systems struggle with cold starts, which occur when new users or items are added to the system. For example:

    New users: Without historical preferences (e.g., likes, ratings, or interactions), the system has little data to generate meaningful recommendations.
    New items: Items without prior engagement data (e.g., reviews or ratings) cannot be linked effectively to existing users or similar items, leading to poor recommendations.
    This limitation disrupts the system's ability to provide accurate and relevant suggestions for both new users and items.
       Also, recommender systems often expose users to the same views, perspectives, and ideas based on their past interactions. This can lead to:
    Filter bubbles: Users are repeatedly recommended content that aligns with their existing preferences, limiting exposure to diverse or opposing viewpoints.
    Bias reinforcement: Particularly in sensitive areas like politics or science, recommendations may consistently reinforce pre-existing biases, deepening polarization.
    Ethical concerns: The lack of diversity in recommendations hinders opportunities for personal growth, learning, and exploring new perspectives, raising ethical questions about the system's impact on society.
   

3. Transfer learning in NLP involves using pre-trained models, such as BERT, GPT, or Word2Vec, which have been trained on large datasets to understand general language patterns, and fine-tuning them on specific tasks with new data. This approach is highly efficient because it avoids the computational expense of training models from scratch, especially when dealing with vast vocabularies like in English. These pre-trained models provide embeddings or contextual representations of language that can be applied to tasks like text classification or sentiment analysis. However, they may also inherit societal biases present in the original training data, which can impact fairness and accuracy in downstream applications.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)